In [104]:
# importing required libraries
import pandas as pd
import requests
import nbimporter
from win_prediction_functions import get_player_data, get_player_stats, player_stats_calc, calculate_match_stats
from auth_key import get_auth_key

In [106]:
auth_key = get_auth_key()

In [108]:
def new_player_data(team_a_nicknames, team_b_nicknames, map, auth_key):

    # creating columns for new data frame
    columns = ["map", "Team_A_avg_win_percentage", "Team_A_avg_KR", "Team_A_avg_elo",
                  "Team_B_avg_win_percentage", "Team_B_avg_KR", "Team_B_avg_elo"]
    
    # creating match data frame
    
    data_df = pd.DataFrame(columns = columns)
    data_df.columns

    # assigning none to Win row and assigning map
    data_df.loc[0, "map"] = map

    # Team A Stats
    team_a_id_elo_list = []

    # loops over nicknames column and appends list of player ids for team A
    for nickname in team_a_nicknames:
        id_elo = get_player_data(nickname, auth_key)
        
        # returns empty dataframe if player elo DNE
        if id_elo.empty:
            return id_elo
        else:
            team_a_id_elo_list.append(id_elo)

    # creates column of player IDs from list
    team_a_id_elo_df = pd.concat(team_a_id_elo_list)

    # adds mean of team elo to match data df
    data_df.loc[0 , "Team_A_avg_elo"] = team_a_id_elo_df["faceit_elo"].mean()
    
    # Uses player ID to calculate individual player stats (K/R and win percentage)
    team_a_stats_list = []
    
    for id in team_a_id_elo_df.player_id:
        # uses get_player_stats to calculate individual stats
        stats_df = get_player_stats(id, auth_key)
        stats = player_stats_calc(stats_df, data_df["map"][0])
        
        # if returned df is empty, return empty match dataframe
        if stats.empty:
            print("Unable to compute stats, may be error with map used")
            return stats

        # append individual stats to list
        else: 
            team_a_stats_list.append(stats)

    # creates win percentage and K/R ratio columns for team A 
    team_a_stats_df = pd.concat(team_a_stats_list)
    data_df.loc[0 , "Team_A_avg_win_percentage"] = team_a_stats_df["Win Percentage"].mean()
    data_df.loc[0 , "Team_A_avg_KR"] = team_a_stats_df["Average K/R Ratio"].mean()
        
    
    
    
    
    # Team B stats, repeats same process used for team A stats
    team_b_id_elo_list = []
    
    for nickname in team_b_nicknames:
        
        if id_elo.empty:
            return id_elo
            
        else:
            team_b_id_elo_list.append(id_elo)
    
    team_b_id_elo_df = pd.concat(team_b_id_elo_list)
    team_b_id_elo_df
    data_df.loc[0 , "Team_B_avg_elo"] = team_b_id_elo_df["faceit_elo"].mean()
    
    
    team_b_stats_list = []
    for id in team_b_id_elo_df.player_id:
        
        stats_df = get_player_stats(id, auth_key)
        stats = player_stats_calc(stats_df, data_df["map"][0])
        
        if stats.empty:
            print("Unable to compute stats, may be error with map used")
            return stats
            
        else: 
            team_b_stats_list.append(stats)
            
    team_b_stats_df = pd.concat(team_b_stats_list)
    data_df.loc[0 , "Team_B_avg_win_percentage"] = team_b_stats_df["Win Percentage"].mean()
    data_df.loc[0 , "Team_B_avg_KR"] = team_b_stats_df["Average K/R Ratio"].mean()

    # Returns row of match data for match_id given
    data_df.drop(columns = ['map'], inplace = True)
    return data_df
    
    

In [110]:
# takes user input and returns list of team_nicknames and map name
def team_inputs():
    # loops over range and takes input for team A player nicknames
    team_a_nicknames = []
    for i in range(1, 6):
        nickname = input(f"Team A nickname {i}:", )
        team_a_nicknames.append(nickname)
    # repeats for team b
    team_b_nicknames = []
    for i in range(1, 6):
        nickname = input(f"Team B nickname {i}:", )
        team_b_nicknames.append(nickname)

    map = input("Map: ",)

    # returns team_a, team_b nicknames and map name
    return team_a_nicknames, team_b_nicknames, map

In [112]:
# calculates win probability given model 
def win_probability_calc(model, auth_key, team_a_nicknames = [], team_b_nicknames = []):
    # using team_inputs and new_player_data func to get player data
    if len(team_a_nicknames) == 0:
        team_a, team_b, map = team_inputs()
        new_data = new_player_data(team_a, 
                                   team_b, 
                                   map, 
                                   auth_key)
    else:
        map = input("Map: ", )
        new_data = new_player_data(team_a_nicknames, 
                                   team_b_nicknames, 
                                   map, 
                                   auth_key)
    print(new_data)
    
    # finding probability given model
    win_prob = model.predict_proba(new_data)

    return win_prob
    

In [114]:
# pulling code from fit and tuning file and fitting a logistic regression model

# readining in win prediction data
data = pd.read_excel("data_win_prediction.xlsx")
data.columns = ["win", "map", 
                "Team_A_avg_win_percentage", 
                "Team_A_avg_KR", "Team_A_avg_elo", 
                "Team_B_avg_win_percentage", 
                "Team_B_avg_KR", "Team_B_avg_elo", 
                "Match ID"]

# checking for missing data
data.isna().sum()

# checking for duplicated rows
data.duplicated().sum()

# removing duplicated rows
data = data.drop_duplicates()
data.duplicated().sum()

# spliting features and response variable
features = data.loc[:, ["Team_A_avg_win_percentage", "Team_A_avg_KR", "Team_A_avg_elo",
                  "Team_B_avg_win_percentage", "Team_B_avg_KR", "Team_B_avg_elo"]]
# transforming response column into a 1d array
response = data.loc[:, ["win"]].values.ravel()

# Importing required models for logistic regession
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold

# fitting the regression model
lr_model = LogisticRegression(max_iter = 200)

# splitting data into k folds
folds = KFold(n_splits = 10, shuffle = True, random_state = 50)

# Perform k-folds cross validation
scores = cross_val_score(lr_model, features, response, cv = folds, scoring = 'accuracy')
scores.mean()

# fitting model 
fit_model = lr_model.fit(features, response)

In [140]:
team_a_nicknames  = ["WOAH234", "Gemini-JC", "Fraqment", "SinistroxX", "Sir_cosmin"]
team_b_nicknames = ["Skrappi", "SwervinErvin", "JerbaTerba", "Briney78", "Dark_SZ"]

In [136]:
fit_model = lr_model.fit(features, response)

In [142]:
win_probability_calc(fit_model, auth_key, team_a_nicknames = team_a_nicknames, team_b_nicknames = team_b_nicknames)

Map:  de_mirage


ZeroDivisonError
Unable to compute stats, may be error with map used
Empty DataFrame
Columns: []
Index: []


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


ValueError: at least one array or dtype is required

In [66]:
new_player_data(team_a_nicknames, team_b_nicknames, "de_inferno", auth_key)

,Team_A_avg_win_percentage,Team_A_avg_KR,Team_A_avg_elo,Team_B_avg_win_percentage,Team_B_avg_KR,Team_B_avg_elo
0,29.906417,0.6367,909.2,0.0,0.41,943.0
